# 6.4 モジュール

**モジュール**(module)は、クラスのように、メソッド・定数・クラス変数のグループに名前を付けたものである。モジュールはクラスと同じように定義されるが、classキーワードではなく、**module**キーワードが使われる。しかし、クラスとは異なり、モジュールはインスタンスを作ることはできないし、サブクラス化することもできない。モジュールは独立しており、継承の「モジュール階層」のようなものはない。モジュールは、名前空間として、またミックスインとして使われる。本章では、この２つの使い方を説明していく。

## 名前空間としてのモジュール

モジュールは、オブジェクト指向プログラミングが不要なときに、関連するメソッドをグループにまとめる方法として優れている。たとえば、数学のべき乗や三角関数を求める計算の計算結果は、引数のみに依存する。このように状態を取る必要がない処理にはオブジェクト指向をする必要がない。その場合は次のように、グローバルメソッドを定義すれば良い。

In [40]:
module MyMath
    PI = 3.14159265359
    
    def self.abs(n)
        n < 0 ? -n : n
    end
    
    def self.pow(x, n)
        (1..n-1).reduce(x){|f, _| f * x}
    end
end

:pow

モジュールを定義することで、インスタンス化すること無く、次のようにモジュールのメソッドを呼び出すことができる。

In [41]:
MyMath.abs(-4)

4

In [42]:
MyMath.pow(4, 3)

64

モジュールでは、定数を定義することもできる。定数は接頭辞として、```モジュール名+(::)```を付けて、その後に定数名を付けることで呼び出すことが出来る。

In [44]:
MyMath::PI

3.14159265359

## ミックスインとしてのモジュール

前章でRubyにおける継承を学んだ。Rubyを始め継承の概念を持つ多くの言語は、複数のスーパークラスを継承すること「**多重継承**」を許していない。多重継承を許さない理由として、ある子クラス*D*が複数のスーパークラス(*B*・*C*)が同一の親(*A*)を持ち(Rubyでは、ダックタイピングがあるため親を持たなくても良い)ある一つのメソッドをオーバーライドするとき、子クラス*D*がそのメソッドを呼出したとき、B・Cどちらのメソッドを呼び出すか曖昧になってしまうからである。この問題は、[菱形継承問題](https://ja.wikipedia.org/wiki/菱形継承問題)として一般に知られている。また、継承による階層構造が極端に複雑になってしまうという問題も抱えている。

```ruby
class Sub < Super1, Super2 # Rubyでは、シンタックスエラー。
end
```

Rubyでは多重継承を擬似的に再現するために、モジュールによる**ミックスイン**(Mixin)を行う。ミックスインとは、クラスやモジュールに、モジュールをトッピングするような形で機能を追加していくことである。ミックスインは、継承と違いモジュールをいくつでも合成することが出来る。菱形継承問題は、**階層構造を線形化**することで解決している。線形化については後で説明をする。以下にモジュールのミックスインの例を示す。

In [7]:
class BasicStack
  attr_reader :stack

  def initialize(stack=[])
    @stack = stack.freeze
  end

  def push(v)
    self.class.new(@stack.dup.concat([v]))
  end

  def pop
    self.class.new(@stack[0..-2])
  end
end

module Doubling
  def push(v)
    super(v * 2)
  end
end

module PlusTwo
  def push(v)
    super(v + 2)
  end
end

class MyStack < BasicStack
  include PlusTwo
  include Doubling
end

MyStack

ミックスインを行わない*BasicStack*クラス。

In [8]:
BasicStack.new.push(1).push(2).push(3).push(4)

#<BasicStack:0x0055a9ba2d2468 @stack=[1, 2, 3, 4]>

ミックスインを行った*MyStack*クラス。MyStackは、BasicStackの継承に加え、*PlusTwo*・*Doubling*クラスをミックスインしている。この場合の継承・ミックスインの階層構造は、最後にミックスインされたモジュールから最初のモジュールまで逆順。そして、スーパークラスという順番で線形に解決される。つまり、

- MyStack::push     存在しないため、ミックスインモジュールを探索
- Doubling::push    vを2倍、ミックスインモジュールを探索
- PlugTwo::push     vをプラス2、ミックスインモジュールを探索 -> 存在しない為サブクラスを探索
- BasicStack::push  @stackにvを格納、クラス(MyStack)をインスタンス化する。

という順番で呼び出される。もし、*BasicStack*がpushメソッドを持たない場合は、さらにスーパークラスを探索し、最終的にmissing_method(エラー)を呼び出す。

In [9]:
MyStack.new.push(1).push(2).push(-1).push(3).push(4)

#<MyStack:0x0055a9ba277950 @stack=[4, 6, 0, 8, 10]>

クラスの継承・ミックスインの階層構造を確認するには、*ancestors*メソッドを呼び出す。

In [12]:
MyStack.ancestors

[MyStack, Doubling, PlusTwo, BasicStack, Object, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

ミックスインは、柔軟にクラスを拡張する手段である。ミックスイン機能を持った有名な言語は、Ruby・PHP・Scalaなどが存在する。ここまでに、クラスを拡張する方法を、いくつか紹介した。クラスの拡張を考える際は、以下のような順序で拡張を考えるのが一つので手である。

- HAS-Aの関係でコンポジションを考える。
- ミックスインでモジュールによる機能を追加を考える。
- IS-Aの関係が成り立つのを確認してから、継承を考える。

## 演習

- MyStackのミックスインの順序を変えて、挙動を確認せよ。
- 負の値をスタックに格納しない(フィルタリング)、新しいモジュール*NegativeFilter*を加えて、ミックスインしろ。

## チェックポイント

- モジュールはどのような使い方があるか
- 継承とミックスインの違いは何か